In [1]:
from pyshacl.validate import Validator
from rdflib import OWL, RDF, RDFS, SH, Graph, Namespace

In [2]:
S223 = Namespace("http://data.ashrae.org/standard223#")

In [3]:
g = Graph()
g.parse('../223standard/models/MODEL_SP223_core-v1.0.ttl')

<Graph identifier=Nbedbfdc3e60a444292b49b7f38d1767d (<class 'rdflib.graph.Graph'>)>

In [5]:
# Validation function for G36
def run_validation(g, sg):
    v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
    conforms, report_graph, report_text = v.run()


    namespace_map = {}
    for prefix, uriref in report_graph.namespaces():
        namespace_map[prefix] = Namespace(uriref)
        # Need OR for result Message
    qs = """
        SELECT ?focusNode ?resultMessage ?resultSeverity ?resultPath
        WHERE {
            BIND(s223:g36 AS ?resultSeverity) .
            ?report rdf:type sh:ValidationReport .
            ?report sh:result ?result .
            ?result sh:focusNode ?focusNode .
            OPTIONAL {?result sh:resultMessage ?resultMessage} .
            ?result sh:resultPath ?resultPath .
            ?result sh:resultSeverity ?resultSeverity .
            }
        """

    # pretty colors
    color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35, S223.g36: 36}

    # query
    results = sorted(report_graph.query(qs, initNs=namespace_map))
    
    prev = None
    for focusNode, resultMessage,resultSeverity, resultPath in results:
        if focusNode != prev:
            print(focusNode)
            prev = focusNode
        color = color_map[resultSeverity]
        print(f"\x1b[{color}m    {resultMessage}\x1b[0m   {resultPath}\x1b[0m")
    return report_text

# g36 VAV Cooling Only

In [26]:
g = Graph ()
g.parse('vav41.ttl')

#add model files from 223p repository if needed (input local paths and uncomment)
#g.parse('../../223standard/models/MODEL_SP223_system-v1.0.ttl')


sg = Graph()
sg.parse('g36vav41.ttl')

# Add main model files from s223 repository if needed
# sg.parse('../../223standard/models/MODEL_SP223_core-v1.0.ttl')
# sg.parse('../../223standard/models/MODEL_SP223_device-v1.0.ttl')
# sg.parse('../../223standard/models/MODEL_SP223_system-v1.0.ttl')

<Graph identifier=N10e98ee6438b476aa3e3970e9ed3e08a (<class 'rdflib.graph.Graph'>)>

In [27]:
# Running validation, showing only when G36 rules have been violated.
report_text = run_validation(g,sg)

http://data.ashrae.org/standard223/data/g36-figure-a-1#00019
    No Discharge Flow Property, try using vav-flow property instead   http://data.ashrae.org/standard223#hasSystemConnectionPoint


In [28]:
'No Discharge Flow Property' in report_text

True

In [29]:
# Can do minor changes to rules using SPARQL queries for testing
# Here I am deactivating the discharge flow property shape, and looking for vav-flow instead
update = """ 
    DELETE DATA {
        :s223-discharge-flow sh:deactivated false .
    } """
update2 = """
    INSERT DATA {
        :s223-discharge-flow sh:deactivated true .
    }
"""

In [30]:
# Altering shacl rules using updates
sg.update(update)
sg.update(update2)

In [31]:
report_text = run_validation(g,sg)

In [32]:
'No Discharge Flow Property' in report_text

False

## Trying out Flow Or and sh:detail
This is random experimentation to get experience writing shacl shapes works. Here I'm testing out using sh:and and sh:or on a simple model file that I've created

In [23]:
g = Graph ()
g.parse('sh-and-or-test.ttl')


sg = Graph()
sg.parse('sh-and-or-test.ttl')

<Graph identifier=Nb687f0c9bb774c4ba54e75bbfe3aac75 (<class 'rdflib.graph.Graph'>)>

In [24]:
v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
conforms, report_graph, report_text = v.run()

In [25]:
print(report_text)

Validation Report
Conforms: False
Results (2):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:constraint
	Focus Node: test:fail_and_succeed_or
	Value Node: test:fail_and_succeed_or
	Message: top level constraint
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:constraint
	Focus Node: test:fail_or_succeed_and
	Value Node: test:fail_or_succeed_and
	Message: top level constraint

